In [1]:
import sys
sys.path.append("../") 

In [2]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

# Load data from https://www.openml.org/d/554
X, y = fetch_openml('mnist_784', version=1, return_X_y=True, data_home='../data',cache=True,as_frame=False)
X=X/255.

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2)

In [6]:
import numpy as np
from hamiltonian import utils

num_examples,num_inputs=X_train.shape
num_outputs=len(np.unique(y_train))

#y_train=utils.one_hot(y_train,num_outputs)
#y_test=utils.one_hot(y_test,num_outputs)

In [7]:
y_train=np.int32(y_train)
y_test=np.int32(y_test)

In [51]:

import mxnet as mx
from mxnet import nd, autograd, gluon


model_ctx=mx.cpu()
par={'weights':nd.random_normal(shape=(num_inputs,num_outputs),ctx=model_ctx),'bias':nd.random_normal(shape=num_outputs,ctx=model_ctx)}
hyper={'alpha':nd.array(20.0, ctx=model_ctx)}

In [52]:
from hamiltonian.inference.bbb import bbb
from hamiltonian.models.softmax import softmax

model=softmax(hyper,ctx=model_ctx)
inference=bbb(model,par,step_size=0.05,ctx=model_ctx)

In [53]:
import importlib
import hamiltonian
importlib.reload(hamiltonian.models.softmax)
importlib.reload(hamiltonian.inference.bbb)

<module 'hamiltonian.inference.bbb' from '../hamiltonian/inference/bbb.py'>

In [ ]:
par,loss=inference.fit(epochs=10,batch_size=64,X_train=X_train,y_train=y_train,verbose=False)

 30%|███       | 3/10 [00:05<00:11,  1.68s/it]

In [ ]:
import matplotlib.pyplot as plt

plt.plot(loss)
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()

In [ ]:
loss

In [29]:
y_pred=model.predict(par,nd.array(X_test,ctx=model_ctx))

In [30]:
samples=[]
for _ in range(100):
    samples.append(y_pred.sample().asnumpy())

In [16]:
samples=np.asarray(samples)

In [17]:
samples.shape

(100, 14000)

In [18]:
y_hat=np.quantile(samples,.5,axis=0)

In [19]:
from sklearn.metrics import classification_report

print(classification_report(y_test,np.int32(y_hat)))

              precision    recall  f1-score   support

           0       0.94      0.94      0.94      1405
           1       0.94      0.94      0.94      1557
           2       0.88      0.84      0.86      1394
           3       0.86      0.85      0.85      1432
           4       0.88      0.88      0.88      1378
           5       0.80      0.82      0.81      1226
           6       0.92      0.93      0.92      1399
           7       0.88      0.91      0.90      1436
           8       0.79      0.81      0.80      1343
           9       0.85      0.83      0.84      1430

    accuracy                           0.88     14000
   macro avg       0.87      0.87      0.87     14000
weighted avg       0.88      0.88      0.88     14000



In [ ]:
X_batch=nd.array(X_train[:32])
y_batch=nd.array(y_train[:32])

In [ ]:
like=model.negative_log_likelihood(par,X_train=X_batch,y_train=y_batch)
prior=model.negative_log_prior(par)

In [ ]:
like

In [ ]:
prior

In [ ]:
like+prior